In [1]:
import json 
from stackapi import StackAPI
from magi_dataset import *
from tqdm import tqdm
import time
import os

# create ./data dir to cache dataset
if not os.path.exists('./data'):
    os.mkdir("./data")

# github dataset with 5000 repo
github_dataset = GitHubDataset(
    empty = False
)


  full_bar = Bar(frac,
magi_dataset.magi_dataset - INFO - Loaded 5000 repos from C:\Users\Junqi Chen\AppData\Local\Temp\tmp8slkm1fm\ghv9-2.json


In [2]:
# Proxy Engine (ignore what it does)
from bs4 import BeautifulSoup
import requests

useful_proxy = {}
i = 0

def get_proxy():
    proxies = [] # proxies poll
    proxy = {} # now using proxy
    
    # load the proxies poll
    proxies.clear()
    url='https://sslproxies.org/'
    response=requests.get(url)
    soup=BeautifulSoup(response.content, 'lxml')
    for item in soup.select('table[class="table table-striped table-bordered"] > tbody > tr'):
        try:
            proxies.append({'ip': item.select('td')[0].get_text(), 
                            'port': item.select('td')[1].get_text()})
        except:
            pass
    
    # random select a current proxy
    global i
    if i >= len(proxies):
        i = 0
    randomIP = proxies[i]['ip']
    randomPort = proxies[i]['port']
    proxy =  {"https": f"https://{randomIP}:{randomPort}"}
    i += 1
    print(f"now using proxy {proxy}")
    return proxy

def repeat_get():
    global useful_proxy
    if useful_proxy:
        current_proxy = useful_proxy
    else:
        current_proxy = get_proxy()
    SITE = None
    while not SITE:
        try:
            SITE = StackAPI('stackoverflow', proxy=current_proxy)
            if not useful_proxy and SITE: # useful_proxy is empty and response get
                useful_proxy = current_proxy
        except:
            print(f"discard current useful proxy {current_proxy}.")
            useful_proxy = {}
            current_proxy = get_proxy()
    return SITE

In [3]:
stackoverflow_dataset = []
SITE = repeat_get()

for i, repo in enumerate(tqdm(github_dataset)):
    if i < 4516:
        continue
    repo_name = repo.name
    repo_dict = {"name":repo_name, "item":[]}
    url = f'https://github.com/{repo_name}'

    # Search the question list given the url and sort based on activity
    try:
        question_list = SITE.fetch('search/excerpts', url = url, sort='activity', order='desc')
    except:
        print(f"SITE Error occurs with dataset length {len(stackoverflow_dataset)}.")
        print("Saving current data...")
        # Save to json file when error occurs
        filename = './data/stackoverflow_dataset.json'
        if os.path.isfile(filename) is False:
            with open(filename, 'w') as outfile:
                json.dump(stackoverflow_dataset, outfile, indent=2)
        else:
            with open(filename) as infile:
                data = json.load(infile)
            data += stackoverflow_dataset
            with open(filename, 'w') as outfile:
                json.dump(data, outfile, indent=2)
        # Handle SITE Error
        SITE = repeat_get()
        question_list = SITE.fetch('search/excerpts', url = url, sort='activity', order='desc')
        stackoverflow_dataset = []

    # Store the top 5 questions of each question to the dictionary 
    for question in question_list['items'][:5]:
        question_title = question['title']
        repo_dict['item'].append(question_title)
    stackoverflow_dataset.append(repo_dict)
#     time.sleep(0.5)


now using proxy {'https': 'https://18.166.72.199:8081'}
discard current useful proxy {'https': 'https://18.166.72.199:8081'}.
now using proxy {'https': 'https://112.217.162.5:3128'}
discard current useful proxy {'https': 'https://112.217.162.5:3128'}.
now using proxy {'https': 'https://103.28.100.11:3128'}


  0%|                                                                                         | 0/5000 [00:00<?, ?it/s]

SITE Error occurs with dataset length 0.
Saving current data...
discard current useful proxy {'https': 'https://103.28.100.11:3128'}.
now using proxy {'https': 'https://18.166.72.199:8081'}
discard current useful proxy {'https': 'https://18.166.72.199:8081'}.
now using proxy {'https': 'https://112.217.162.5:3128'}
discard current useful proxy {'https': 'https://112.217.162.5:3128'}.
now using proxy {'https': 'https://103.28.100.11:3128'}
discard current useful proxy {'https': 'https://103.28.100.11:3128'}.
now using proxy {'https': 'https://188.0.147.102:3128'}
discard current useful proxy {'https': 'https://188.0.147.102:3128'}.
now using proxy {'https': 'https://31.186.239.245:8080'}
discard current useful proxy {'https': 'https://31.186.239.245:8080'}.
now using proxy {'https': 'https://104.223.135.178:10000'}
discard current useful proxy {'https': 'https://104.223.135.178:10000'}.
now using proxy {'https': 'https://117.251.103.186:8080'}


 96%|███████████████████████████████████████████████████████████████████████████▏  | 4816/5000 [18:15<08:34,  2.79s/it]

SITE Error occurs with dataset length 300.
Saving current data...
discard current useful proxy {'https': 'https://117.251.103.186:8080'}.
now using proxy {'https': 'https://5.9.149.118:40000'}


...

 97%|█████████████████████████████████████████████████████████████████████████▍  | 4835/5000 [23:43<1:45:01, 38.19s/it]

SITE Error occurs with dataset length 1.
Saving current data...


 97%|███████████████████████████████████████████████████████████████████████████▋  | 4851/5000 [24:44<04:47,  1.93s/it]

SITE Error occurs with dataset length 16.
Saving current data...


100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [32:56<00:00,  2.53it/s]


In [12]:
filename = './data/stackoverflow_dataset.json'
data = []
with open(filename) as infile:
    temp = json.load(infile)
    
for i, d in enumerate(temp):
    if i < 135:
        data.append(d)
    else:
        data += d
        
with open('./data/stackoverflow_real.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)